# UI to conduct a manual audit of the labels from conventions classifiers. 


# ***IF NOT USING BINDER - UNCHECK THE BOX***


The output of the classifiers are stored to

    -file 'audit_training_data.gz (which is a zipped pandas database) 
    -location: cp_wssc/Data/Iterative-models-building/Classification results/Conventions/

The UI randomly displays an already classified sentence for the purpose of audit and correction

Changes made will be stored locally until the "Save to GIT" button is selected, in which case the modified pandas dataframe will be zipped and stored to 

    - file 'audit_training_data.gz (which is a zipped pandas database)
    -location: cp_wssc/Data/Iterative-models-building/Classification results/Conventions/

Overriding the original file at which point the audit of the labeling may continue


In [1]:
import os
import re
import random
import glob
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display, clear_output, HTML
import base64
import io
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import base64
from github import Github
from github import InputGitTreeElement
import requests
import time

In [2]:
%%javascript
//To remove scroll from any output area and automatically extend the jupyter cell
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [14]:
FLAG__ON_BINDER = True
FLAG__FROM_GIT = True

my_style = {'description_width': 'initial'}
my_layout = {'width': '500px'}
my_layout_short = {'width': '300px'}


box_binder = widgets.Checkbox(True, description='*ONLY* CLICK THIS BOX IF RUNNING ON BINDER', style = my_style, layout = my_layout)

def update_box_binder(change):
    global FLAG__ON_BINDER
    global FILE_PATH
    global DOWNLOAD_PATH
    FLAG__ON_BINDER = change['new']
    
    if (FLAG__ON_BINDER == True):
        DOWNLOAD_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'
        DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
        FILE_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'

    else:
        DOWNLOAD_PATH = './Data/Iterative-models-building/Classification results/Conventions/'
        DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
        FILE_PATH = './Data/Iterative-models-building/Classification results/Conventions/'
    print (FILE_PATH)
   
    
box_binder.observe(update_box_binder, 'value')

display(box_binder)


Checkbox(value=True, description='*ONLY* CLICK THIS BOX IF RUNNING ON BINDER', layout=Layout(width='500px'), s…

./Data/Iterative-models-building/Classification results/Conventions/
False
/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/
True
./Data/Iterative-models-building/Classification results/Conventions/
False


In [4]:
FILE_NAME = 'audit_training_data.gz'

TOTAL_RECORDS_AUDITED = 0
TOTAL_UNSAVED_AUDITS = 0
LAST_SAVED = ""

if FLAG__ON_BINDER == True:
    DOWNLOAD_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'
    DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
    FILE_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'
    
else:
    DOWNLOAD_PATH = './Data/Iterative-models-building/Classification results/Conventions/'
    DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
    FILE_PATH = './Data/Iterative-models-building/Classification results/Conventions/'
    
if FLAG__FROM_GIT == True:
    DOWNLOAD_PATH = 'https://github.com/aideenf/AIVC/blob/master/cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/audit_training_data.gz?raw=true'
    GIT_PUSH_PATH = 'cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'
    

In [5]:
def retrieve_data_frame ():
   
    if FLAG__FROM_GIT == True:
        # sep='\t'  to specify tab seperated?
        df = pd.read_csv(DOWNLOAD_PATH, error_bad_lines=False, compression='gzip', header=0, quotechar='"')

    if FLAG__FROM_GIT == False:
        df = pd.read_csv(DOWNLOAD_PATH, sep='\t')

    df = df.drop(df.columns[0], axis=1)
    #print("num of records with NaN", df.isna().sum() )
    #Drop any rows where there is 0 confidence value. 
    df = df.dropna(subset=['confidence_value'])
    #if source is not specified, fill as other. 
    df['data_provenance'] = df.data_provenance.fillna('other')
     #print("num of records with NaN", df.isna().sum() )
    row_to_edit = df.sample(n = 1)
    index_number = 0 
    print("data file loaded (or re-loaded after save to git)")
    return df


In [6]:
convention_sentences_df = []

audited_df = pd.DataFrame(columns=['text', 'provenance', 'convention', 'old', 'new'])
convention_sentences_df = retrieve_data_frame ()

data file loaded (or re-loaded after save to git)


In [7]:
min_confidence = convention_sentences_df.confidence_value.min()
if min_confidence < .01:
    min_confidence = 0

max_confidence = convention_sentences_df.confidence_value.max()
display ("Data to audit loaded from GitHub")
display (convention_sentences_df.head(1))

'Data to audit loaded from GitHub'

,text,confidence_value,confidence_level,data_provenance,convention_civic,convention_domestic,convention_green,convention_industrial,convention_inspired,convention_market,convention_project,convention_renown
0,Provides easily configurable data import progr...,0.00039,Low,efficiency,0,1,0,0,0,0,0,0


In [25]:
def time_stamp():
    now = datetime.now()
    timestamp = datetime.timestamp(now)
    dt_object = datetime.fromtimestamp(timestamp)
    words = str(dt_object).split(' ');
    return words[0], words[1]

def save_to_github(git_user, git_password, git_repo, my_file_list, push_to_git_as):
    '''
    in order to push a file to github it must first be stored locally, then pushed
    this local location can also be local to a virtual machine. 
    takes: 
            git username, password, repo, 
            a list of files to push to git ie the full local location of file,
            a matching list of paths to push each file to in Git hub 
    '''
    user = git_user
    password = git_password
    url = git_repo
    file_list = []  #push these list of files to git
    file_names = [] #push to this location in git
    
    
    try:
        g = Github(user, password)
        repo = g.get_user().get_repo(url)

        file_list = my_file_list 
        file_names = push_to_git_as
                 
        commit_message = 'training data updated via the audit tool'
    
        master_ref = repo.get_git_ref('heads/master')
        master_sha = master_ref.object.sha
        base_tree = repo.get_git_tree(master_sha)
        element_list = list()

        for i, entry in enumerate(file_list):
            print ("file to commit:", entry)
            
            with open(entry) as input_file:
                #data = input_file.read()   #works with non zip file
                data = base64.b64encode(open(entry, "rb").read())
                
            if entry.endswith('.png'):
                data = base64.b64encode(data) 
                

            blob = repo.create_git_blob(data.decode("utf-8"), "base64")
            element = InputGitTreeElement(path=file_names[i], mode='100644', type='blob', sha=blob.sha)

            #push to git as file_names[i]
            print ("push to git as:", file_names[i])
            
            #element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
            
            #element_list is a list of InputGitTreeElement. 
            #Each one corresponds to a file. 
            # the 'content' of InputGitTreeElement can only be of type 'str' or 'unicode'. 
            #When I load a file to memory I have type 'bytes'. 
            #What is the right way to encode those bytes to str or unicode to upload a .zip
            element_list.append(element)   
        
        tree = repo.create_git_tree(element_list, base_tree)
        parent = repo.get_git_commit(master_sha)
        commit = repo.create_git_commit(commit_message, tree, [parent])
        master_ref.edit(commit.sha)
        return commit
    except e:
        print("")
        print ("GITHUB SUBMIT FAILED:")
        print ("Are your github login credentials correct?")
        print ("Are you a collaberator in the repo?")
        print(e)
        return e


def save_to_github_not_zip(git_user, git_password, git_repo, my_file_list, push_to_git_as):

    user = git_user
    password = git_password
    url = git_repo
    file_list = []  #push these list of files to git
    file_names = [] #push to this location in git
    
    try:
        g = Github(user,password)
        repo = g.get_user().get_repo(url)

        file_list = my_file_list 
        file_names = push_to_git_as
                 
        commit_message = 'training data updated via the audit tool'
    
        master_ref = repo.get_git_ref('heads/master')
        master_sha = master_ref.object.sha
        base_tree = repo.get_git_tree(master_sha)
        element_list = list()

        for i, entry in enumerate(file_list):
            with open(entry) as input_file:
                data = input_file.read()
            if entry.endswith('.png'):
                data = base64.b64encode(data)
                
            print ("file to commit:", entry)
            print ("push to git as:", file_names[i])
            element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
            element_list.append(element)
            
        tree = repo.create_git_tree(element_list, base_tree)
        parent = repo.get_git_commit(master_sha)
        commit = repo.create_git_commit(commit_message, tree, [parent])
        master_ref.edit(commit.sha)
        print (commit)
    except:
        print("")
        print ("GITHUB SUBMIT FAILED:")
        print ("Are your github login credentials correct?")
        print ("Are you a collaberator in the repo?")



In [36]:


selectHTML = widgets.HTML(
            "<h3><left>Apply Audit Filter</left></h3>")


saveHTML = widgets.HTML(
            "<h4><left>push changes to github</left></h4>")

space = widgets.Label('  ', layout=widgets.Layout(width='100%'))


convention_drop_down = widgets.Dropdown(
    options=['convention_civic', 'convention_domestic', 'convention_green', 'convention_industrial', 'convention_inspired', 'convention_market', 'convention_project', 'convention_renown'],
    value='convention_civic',
    description='Convention:',
    disabled=False,
    style = my_style,
    layout = my_layout
)

source_drop_down = widgets.Dropdown(
    options =  convention_sentences_df.data_provenance.unique(),
    description='Data source:',
     #value=None,
    disabled=False,
    style = my_style,
    layout = my_layout
)



reclassify_drop_down = widgets.Dropdown(
    options=['flip current', 'convention_civic', 'convention_domestic', 'convention_green', 'convention_industrial', 'convention_inspired', 'convention_market', 'convention_project', 'convention_renown'],
    value='flip current',
    description='Re-Classify:',
    disabled=False,
    style = my_style,
    layout = my_layout
)



convention_true_false = widgets.Dropdown(
    options=['True', 'False'],
    value='True',
    description='Logged as:',
    disabled=False,
    style = my_style,
    layout = my_layout
)

confidence_score_slider_min = widgets.FloatSlider(
    min=min_confidence, 
    max=max_confidence,
    value = 0,
    step = 0.1,
    description='Min Confidence level:',
    style = my_style,
    layout = my_layout
    )


confidence_score_slider_max = widgets.FloatSlider(
    min=min_confidence, 
    max=max_confidence, 
    value = max_confidence,
    step = 0.1,
    description='Max Confidence level:',
    style = my_style,
    layout = my_layout
    )

change_value_button = widgets.Button(
            description='Re-Classify',
            disabled=False,
            button_style='',
            tooltip='Click me',
        )


keep_value_button = widgets.Button(
            description='Classified OK',
            disabled=False,
            button_style='',
            tooltip='Click me',
        )

audit_next_button = widgets.Button(
            description='Skip',
            disabled=False,
            button_style='',
            tooltip='Click me',
        )


save_to_git_button = widgets.Button(
            description='Save to GitHub',
            disabled=False,
            button_style='',
            tooltip='Click me',
        )




user = widgets.Text(
            description='GIT username',
            disabled=False,
            layout=my_layout_short,
            tooltip='enter the new value name',
            style = my_style
        )


pswd = widgets.Password(
            description='GIT password',
            disabled=False,
            layout=my_layout_short,
            tooltip='enter the new value name',
            style = my_style
        )



def change_value_button_clicked(b):
    global TOTAL_UNSAVED_AUDITS 
    global TOTAL_RECORDS_AUDITED 
    global index_number
    global row_to_edit
    global audited_df
    global convention_sentences_df
    
    with output:
        current_value = row_to_edit[convention_drop_down.value].values.astype(int)[0]
        if (current_value == 1) :
            new_value = 0
            print ("Changed from True to False(", new_value,")")
        elif (current_value == 0):
            new_value = 1
            print ("Changed from False to(", new_value,")")
            
        #convention_sentences_df[convention_drop_down.value][row_to_edit.index.values.astype(int)[0]] = new_value
        #Add info to the dataframe of Audited
        
        #audited_df = pd.DataFrame(columns=['text', 'provenance', 'convention'])
        text = row_to_edit.text.values.astype(str)[0]
        prov = row_to_edit.data_provenance.values.astype(str)[0]
        conv = convention_drop_down.value.replace('convention_', '')
        
        if (reclassify_drop_down.value != 'flip current'):
            reclass = reclassify_drop_down.value.replace('convention_', '')
            new_value = current_value
            current_value = 0
        else:
            reclass = conv
        
        #audited_df = pd.DataFrame(columns=['text', 'provenance', 'convention', 'old', 'new'])
        current = pd.DataFrame({'text': [text],
                                'provenance': [prov],
                                'convention': [reclass],
                                'old': [current_value],
                                'new': [new_value]
                               })
        audited_df = audited_df.append(current, sort=False, ignore_index=False)
        
        with output:

            print ("change saved locally in data frame")
            TOTAL_UNSAVED_AUDITS = TOTAL_UNSAVED_AUDITS + 1
            TOTAL_RECORDS_AUDITED = TOTAL_RECORDS_AUDITED + 1
            time.sleep(2) 
            clear_output(wait=True)
            output_function(convention_drop_down.value, confidence_score_slider_min.value, confidence_score_slider_max.value, source_drop_down.value )


def keep_value_button_clicked(b):
    global row_to_edit
    global audited_df
    global TOTAL_UNSAVED_AUDITS 
    global TOTAL_RECORDS_AUDITED 
    with output:
        clear_output(wait=True)
        text = row_to_edit.text.values.astype(str)[0]
        prov = row_to_edit.data_provenance.values.astype(str)[0]
        conv = convention_drop_down.value.replace('convention_', '')
        current = pd.DataFrame({'text': [text],
                                'provenance': [prov],
                                'convention': [conv],
                                'old': [1],
                                'new': [1]
                               })
        audited_df = audited_df.append(current,sort=False,ignore_index=False)
        TOTAL_UNSAVED_AUDITS = TOTAL_UNSAVED_AUDITS + 1
        TOTAL_RECORDS_AUDITED = TOTAL_RECORDS_AUDITED + 1
        output_function(convention_drop_down.value, confidence_score_slider_min.value, confidence_score_slider_max.value, source_drop_down.value )


def audit_next_button_clicked(b):
    with output:
        clear_output(wait=True)
        output_function(convention_drop_down.value, confidence_score_slider_min.value, confidence_score_slider_max.value, source_drop_down.value )


def save_changes_to_github_button_clicked(b):
    global TOTAL_UNSAVED_AUDITS 
    global convention_sentences_df
    global LAST_SAVED
    global FILE_PATH
    global GIT_PUSH_PATH
    global audited_df
    GIT_USER = user.value
    GIT_PSWD = pswd.value
    GIT_REPO = 'AIVC'
    D, T = time_stamp()
    file_name = 'audited_training_data_'+ GIT_USER   + '_' + D+T +  '.tsv'
    file_name_zip = 'audit_training_data_'+ GIT_USER + '_' + D+T +   '.gz'
    audited_df.to_csv(FILE_PATH + file_name)
    my_file_list = [FILE_PATH + file_name]
    push_to_git_as = [GIT_PUSH_PATH + file_name]
    commit = save_to_github_not_zip(GIT_USER, GIT_PSWD, GIT_REPO, my_file_list, push_to_git_as)
    if (commit != "error"):
        with outputGitHub:
            clear_output(wait=True)
            print ("File commited to github :", push_to_git_as, "commit:", commit)
            TOTAL_UNSAVED_AUDITS = 0
            D, T = time_stamp()
            LAST_SAVED = D+T
            display(audited_df.head(10))
            display(audited_df.head(10))
            audited_df = pd.DataFrame(columns=['text', 'provenance', 'convention', 'old', 'new'])
    

def output_function(convention_drop_down_value, confidence_score_slider_min_value, confidence_score_slider_max_value, source_drop_down_value ):
    global convention_sentences_df
    global row_to_edit
    global index_number
    global LAST_SAVED
    global TOTAL_RECORDS_AUDITED
    global TOTAL_UNSAVED_AUDIT
    
    reclassify_drop_down.value='flip current'
    convention = convention_drop_down_value.replace('convention_', '')
     
    # Add a filter by > confidence score, maybe this needs to be a slider between two ranges(or two sliders) or a 
    # slider with 3 text values. Low, Middle, High. with low = a to x, middle x to y and high y to z
        
    #df.loc[(df["B"] > 50) & (df["C"] == 900), "A"]
    row_to_edit = convention_sentences_df.loc[(convention_sentences_df[convention_drop_down_value] == 1) &
                                            (convention_sentences_df['data_provenance'] == source_drop_down_value) &
                                            (convention_sentences_df['confidence_value'] >= confidence_score_slider_min_value) &
                                            (convention_sentences_df['confidence_value'] <= confidence_score_slider_max_value)].sample(n = 1)  
    

    
    if not row_to_edit.empty:
        row_to_edit = row_to_edit[['text', convention_drop_down_value, 'confidence_value', 'data_provenance']]
        display (row_to_edit)
        print ("")
        sentence = row_to_edit['text'].values.astype(str)[0]
        display(HTML("<b>Sentence to review: </b><font color='green'>" + sentence + "</font>" ))
        display(HTML("<b>Classified as: </b><font color='green'> " + "true" + "</font> for <font color='green'>" +  convention + " </font>economy of conventions" ))
        index_number = row_to_edit.index.values.astype(int)[0]
        print("Total audited:", TOTAL_RECORDS_AUDITED, " Total unsaved:", TOTAL_UNSAVED_AUDITS," Last Saved:", LAST_SAVED )
        TOTAL_UNSAVED_AUDITS
    else:
        print ("No results, change search setting")
    
    
  
output = widgets.interactive_output(output_function, {
            'convention_drop_down_value': convention_drop_down,
            'confidence_score_slider_min_value': confidence_score_slider_min,
            'confidence_score_slider_max_value': confidence_score_slider_max,
            'source_drop_down_value': source_drop_down
            })

outputGitHub = widgets.Output(layout={'border': '1px solid black'})

change_value_button.on_click(change_value_button_clicked)
keep_value_button.on_click(keep_value_button_clicked)
audit_next_button.on_click(audit_next_button_clicked)
save_to_git_button.on_click(save_changes_to_github_button_clicked)


In [37]:
display (
        space,
        selectHTML,
        convention_drop_down, 
        source_drop_down, 
        confidence_score_slider_min, 
          confidence_score_slider_max, 
        output, 
        space,
        reclassify_drop_down,
        widgets.HBox([change_value_button, keep_value_button, audit_next_button]),
        space,
        saveHTML,
        widgets.HBox([user, pswd, save_to_git_button]),
        space,
        outputGitHub,
        space,
    )

Label(value='  ', layout=Layout(width='100%'))

HTML(value='<h3><left>Apply Audit Filter</left></h3>')

Dropdown(description='Convention:', layout=Layout(width='500px'), options=('convention_civic', 'convention_dom…

Dropdown(description='Data source:', layout=Layout(width='500px'), options=('efficiency', 'HuffPost_News', 'ot…

FloatSlider(value=0.0, description='Min Confidence level:', layout=Layout(width='500px'), max=1.0, style=Slide…

...

HTML(value='<h4><left>push changes to github</left></h4>')

Label(value='  ', layout=Layout(width='100%'))

Output(layout=Layout(border='1px solid black'))

Label(value='  ', layout=Layout(width='100%'))

file to commit: ./Data/Iterative-models-building/Classification results/Conventions/audited_training_data_aideenf_2019-10-0503:33:14.595449.tsv
push to git as: cp_wssc/Data/Iterative-models-building/Classification results/Conventions/audited_training_data_aideenf_2019-10-0503:33:14.595449.tsv
GitCommit(sha="4932387c8a844e54d3ca1117c21ca268661a42f2")
